# Séance 2 - Bonus : ResNet à la main

Pour poursuivre le travail du TP, on se propose d'explorer une autre manière de définir un réseau de neurones au travers d'une architecture classique de Deep Learning pour la vision : les ResNet. 

Commençons par importer et traiter les données.

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

(X_train_full, y_train_full), (X_test, y_test) = (keras.datasets.fashion_mnist.load_data())
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, train_size=0.8)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.astype(np.float32).reshape(-1, 28 * 28)).reshape(-1, 28, 28, 1)
X_valid = scaler.transform(X_valid.astype(np.float32).reshape(-1, 28 * 28)).reshape(-1, 28, 28, 1)
X_test = scaler.transform(X_test.astype(np.float32).reshape(-1, 28 * 28)).reshape(-1, 28, 28, 1)

Pour rappel, une architecture ResNet correspond à la succession de ResBlock qui, dans le papier d'origine, ont l'architecture suivante :
1. Une couche de convolution avec padding pour conserver la taille de l'image
2. Une couche de BatchNormalization, que l'on explicitera en cours à la séance prochaine
3. L'activation ReLU, qui ne doit donc pas être présente dans la couche de convolution
4. Une couche de convolution avec padding pour conserver la taille de l'image
5. Une couche de BatchNormalization
6. Un ajout de l'input avant le point 1 et du résultat de la dernière couche de BatchNormalization (point 5)
7. L'activation ReLU sur l'ajout

On ne peut pas définir cette architecture si l'on utilise la méthode d'instanciation que l'on a utilisé jusqu'ici. Pour y arriver, nous avons besoin d'utiliser la version *fonctionnelle* de Keras. Par exemple avec un modèle dense :

In [ ]:
input = keras.layers.Input(shape=X_train.shape[1:])
flatten = keras.layers.Flatten()(input)
dense_1 = keras.layers.Dense(units=64, activation="relu")(flatten)
dense_2 = keras.layers.Dense(units=64)(dense_1)
activation = keras.layers.ReLU()(dense_2)
output = keras.layers.Dense(units=10, activation="softmax")(activation)

model = keras.models.Model(inputs=[input], outputs=[output])
model.summary()

**Consigne** : En utilisant la version fonctionnelle de Keras, définir une fonction `ResidualBlock` qui prend en paramètre un argument *input* qui correspondra à une couche Keras et en second argument des *kwargs* à passer aux différentes couches de convolutions.

**Consigne** : Définir un modèle utilisant deux ResBlock pour résoudre le problème de classification auquel on s'intéresse.

**Consigne** : Après avoir compilé le modèle, lancer l'entraînement sur quelque époque pour vérifier qu'il fonctionne.

Pour aller plus loin, on pourrait se demander s'il est nécessaire de réaliser ces connexions résiduelles. Pour y répondre, on conseille de construire un réseau de neurones convolutionnel *classique* et comparer les performances pour plusieurs entraînement.